<a href="https://colab.research.google.com/github/atlmapguy/netflix/blob/main/Recommender_Netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pwd

'/content'

In [ ]:
import os

In [ ]:
os.chdir('C:/Users/kevin/Documents/UoL/CSE621')

FileNotFoundError: ignored

In [ ]:
pwd

In [ ]:
conda install -c conda-forge scikit-surprise

In [ ]:
pip install surprise

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD 
from surprise.model_selection import  cross_validate, split
sns.set_style("ticks")

In [ ]:
# Load the data from one of the CSVs and create dataframes
df = pd.read_csv('combined_data_1.txt', header = None, names = ['Id_Cust', 'Rating'], usecols = [0,1])

df['Rating'] = df['Rating'].astype(float)

print('Dataset rows and columns: ' + str(df.shape))
print('We print the first 11 observations to have an idea of how the dataset looks like:')
print(df.iloc[:10])

In [ ]:
df['Rating'].describe()

In [ ]:
df.Rating.describe()

In [ ]:
p = df.groupby('Rating')['Rating'].agg(['count'])

# movie count
count_movie = df.isnull().sum()[1]

# customer count
count_cust = df['Id_Cust'].nunique() - count_movie

# rating count
rating_count = df['Id_Cust'].count() - count_movie

ax = p.plot(kind = 'barh', figsize = (15,10), color= 'black')
plt.title('Total pool: {:,} Movies, {:,} customers, {:,} ratings given'.format(count_movie, count_cust, rating_count), fontsize=20)
plt.axis('on')

for i in range(1,6):
    ax.text(p.iloc[i-1][0]/4, i-1, 'Rating {}: {:.0f}%'.format(i, p.iloc[i-1][0]*100 / p.sum()[0]), color = 'white', weight = 'bold')
    plt.show()

In [ ]:
df['Rating'].describe()

In [ ]:
df_new = pd.DataFrame(pd.isnull(df.Rating))
df_new = df_new[df_new['Rating'] == True]
df_new = df_new.reset_index()

movie_np = []
movie_id = 1

for i,j in zip(df_new['index'][1:],df_new['index'][:-1]):
   
    temp = np.full((1,i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1


last_record = np.full((1,len(df) - df_new.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

print('Movie numpy: {}'.format(movie_np))
print('Length: {}'.format(len(movie_np)))

In [ ]:
df_new

In [ ]:
df = df[pd.notnull(df['Rating'])]

df['Movie_Id'] = movie_np.astype(int)
df['Id_Cust'] = df['Id_Cust'].astype(int)
print('-Dataset examples-')
print(df.iloc[::5000000, :])

In [ ]:
#average number of ppl rated the movies

# count number of ratings per user
users = df.groupby('Id_Cust')['Movie_Id'].nunique()

binwidth = 50
users.hist(bins=np.arange(min(users), max(users) + binwidth, binwidth))    
plt.xlabel('Number of ratings per user')
plt.ylabel('Number of users')
plt.title('Distribution of ratings count per user')

In [ ]:
f = ['count','mean']

df_movie_summary = df.groupby('Movie_Id')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.8),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index

print('Movie minimum times of review: {}'.format(movie_benchmark))

df_cust_summary = df.groupby('Id_Cust')['Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.8),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index

print('Customer minimum times of review: {}'.format(cust_benchmark))

In [ ]:
print('Original Shape: {}'.format(df.shape))
df = df[~df['Movie_Id'].isin(drop_movie_list)]
df = df[~df['Id_Cust'].isin(drop_cust_list)]
print('After Trim Shape: {}'.format(df.shape))
print('-Data Examples-')
print(df.iloc[::5000000, :])

In [ ]:
df

In [ ]:
min_movie_ratings = 10000
filter_movies = (df['Movie_Id'].value_counts()>min_movie_ratings)
filter_movies = filter_movies[filter_movies].index.tolist()

# Filter sparse users
min_user_ratings = 200
filter_users = (df['Id_Cust'].value_counts()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

# Actual filtering
df_filterd = df[(df['Movie_Id'].isin(filter_movies)) & (df['Id_Cust'].isin(filter_users))]
del filter_movies, filter_users, min_movie_ratings, min_user_ratings
print('Shape User-Ratings unfiltered:\t{}'.format(df.shape))
print('Shape User-Ratings filtered:\t{}'.format(df_filterd.shape))

In [ ]:
df_filterd

In [ ]:
pip install -U html5lib=="1.0b10"

In [ ]:
df_title = pd.read_csv('movie_titles.csv', header=None)
df_title.columns = ['Movie_Id', 'Year', 'Name','col_4', 'col_5', 'col_6']
df_title.set_index('Movie_Id', inplace = True)
print (df_title.head(10))

In [ ]:
#Evaluate the performance of the model
reader = Reader()

# We use first 100K rows for faster run time
data = Dataset.load_from_df(df[['Id_Cust', 'Movie_Id', 'Rating']][:120000], reader)

svd = SVD(n_epochs= 20, lr_all= 0.01)
cross_validate(svd, data, measures=['RMSE', 'MAE'])

In [ ]:
reader = Reader()

svd = SVD(n_epochs= 10, lr_all= 0.01)

#Parameters previously found using grid search

user_2043213 = df_title.copy()
user_2043213 = user_2043213.reset_index()
user_2043213 = user_2043213[~user_2043213['Movie_Id'].isin(drop_movie_list)]


data = Dataset.load_from_df(df_filterd[['Id_Cust', 'Movie_Id', 'Rating']], reader)

trainset = data.build_full_trainset()
svd.fit(trainset)

print("SVD Results")

user_2043213['Estimate_Score'] = user_2043213['Movie_Id'].apply(lambda x: svd.predict(2043213, x).est)

user_2043213 = user_2043213.drop('Movie_Id', axis = 1)

user_2043213 = user_2043213.sort_values('Estimate_Score', ascending=False)
print(user_2043213.head(10))

In [ ]:
from surprise import BaselineOnly
#Evaluate the performance of the model
reader = Reader()

# Use first 120K rows for faster run time
data = Dataset.load_from_df(df[['Id_Cust', 'Movie_Id', 'Rating']][:120000], reader)

baselineonly = BaselineOnly()
cross_validate(baselineonly, data, measures=['RMSE', 'MAE'])

In [ ]:
from surprise import CoClustering
#First evaluate the performance of the model
reader = Reader()

# Use first 120K rows for faster run time
data = Dataset.load_from_df(df[['Id_Cust', 'Movie_Id', 'Rating']][:120000], reader)

coclustering = CoClustering()
cross_validate(coclustering, data, measures=['RMSE', 'MAE'])

In [ ]:
from surprise import NMF
#We first evaluate the performance of the model
reader = Reader()

# We use first 120K rows for faster run time
data = Dataset.load_from_df(df[['Id_Cust', 'Movie_Id', 'Rating']][:120000], reader)

nmf = NMF()
cross_validate(nmf, data, measures=['RMSE', 'MAE'])

In [ ]:
from surprise import NMF

nmf = NMF()

user_2649285 = df_title.copy()
user_2649285 = user_2649285.reset_index()
user_2649285 = user_2649285[~user_2649285['Movie_Id'].isin(drop_movie_list)]


data = Dataset.load_from_df(df_filterd[['Id_Cust', 'Movie_Id', 'Rating']], reader)

trainset = data.build_full_trainset()
nmf.fit(trainset)

print("NMF results")

user_2649285['Estimate_Score'] = user_2649285['Movie_Id'].apply(lambda x: nmf.predict(2649285, x).est)

user_2649285 = user_2649285.drop('Movie_Id', axis = 1)

user_2649285 = user_2649285.sort_values('Estimate_Score', ascending=False)
print(user_2649285.head(10))